In [1]:
import json

import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

Função simulada para obter a temperatura de algumas cidades

In [2]:

# (Aqui não há consulta a uma API real, apenas retorna valores fixos para exemplos)
def obter_temperatura_atual(local, unidade="celsius"):
    if "são paulo" in local.lower():
        return json.dumps(
            {"local": "São Paulo", "temperatura": "32", "unidade": unidade}
            )
    elif "porto alegre" in local.lower():
        return json.dumps(
            {"local": "Porto Alegre", "temperatura": "25", "unidade": unidade}
            )
    elif "rio de janeiro" in local.lower():
        return json.dumps(
            {"local": "Rio de Janeiro", "temperatura": "35", "unidade": unidade}
            )
    else:
        return json.dumps(
            {"local": local, "temperatura": "unknown"}
            )

# Define a ferramenta (function) que será disponibilizada para o modelo
tools = [
    {
        "type": "function",
        "function": {
            "name": "obter_temperatura_atual",
            "description": "Obtém a temperatura atual em uma dada cidade",
            "parameters": {
                "type": "object",
                "properties": {
                    "local": {
                        "type": "string",
                        "description": "O nome da cidade. Ex: São Paulo",
                    },
                    "unidade": {
                        "type": "string", 
                        "enum": ["celsius", "fahrenheit"]
                    },
                },
                "required": ["local"],
            },
        },
    }
    ]

# Dicionário que mapeia o nome da função para a função real em Python
funcoes_disponiveis = {
        "obter_temperatura_atual": obter_temperatura_atual,
    }

# Mensagem inicial do usuário para o chatbot
mensagens = [
    {"role": "user", 
     "content": "Qual é a temperatura em São Paulo e Porto Alegre?"}
    ]

# Faz a primeira chamada para o modelo, informando as ferramentas disponíveis
resposta = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=mensagens,
    tools=tools,
    tool_choice="auto",
)
# Extrai a mensagem de resposta do modelo
mensagem_resp = resposta.choices[0].message
tool_calls = mensagem_resp.tool_calls

# Se o modelo solicitou o uso de alguma função (tool_call), executa as funções
if tool_calls:
    # Adiciona a resposta do modelo ao histórico de mensagens
    mensagens.append(mensagem_resp)
    for tool_call in tool_calls:
        # Nome da função a ser chamada
        function_name = tool_call.function.name
        # Função Python correspondente
        function_to_call = funcoes_disponiveis[function_name]
        # Argumentos da função (convertidos de string JSON para dicionário)
        function_args = json.loads(tool_call.function.arguments)
        # Executa a função e obtém a resposta
        function_response = function_to_call(
            local=function_args.get("local"),
            unidade=function_args.get("unidade"),
        )
        # Adiciona a resposta da função ao histórico de mensagens
        mensagens.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )
    # Faz uma segunda chamada ao modelo, agora com as respostas das funções
    segunda_resposta = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=mensagens,
    )

# Extrai a resposta final do modelo e imprime
mensagem_resp = segunda_resposta.choices[0].message
# Exibe a resposta final ao usuário
print(mensagem_resp.content)

A temperatura em São Paulo é de 32°C e em Porto Alegre é de 25°C.
